In [1]:
import pandas as pd 
import numpy as np 
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [6]:
df = pd.read_csv("D:/MachineLearningCourse/Datasets/spam.csv",encoding="latin-1",usecols=["v1","v2"])


In [7]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
df["v1"][df.v1=="ham"] = 0
df["v1"][df.v1=="spam"] = 1

In [9]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df.v2)
X = tokenizer.texts_to_sequences(df.v2)
X = np.array(X)
X_padded = pad_sequences(X,maxlen=35,padding="pre",truncating="pre")

In [10]:
y = df.v1
y  = np.array(y)

In [12]:
x_train = X_padded[:-1000]
y_train = y[:-1000]
x_test = X_padded[-1000:]
y_test = y[-1000:]

In [13]:
print("x_train : ",x_train.shape,"y_train",y_train.shape)
print("x_test : ",x_test.shape,"x_test",y_test.shape)

x_train :  (4572, 35) y_train (4572,)
x_test :  (1000, 35) x_test (1000,)


In [14]:
y_train =np.array(y_train,dtype="int32")
y_test =np.array(y_test,dtype="int32")

In [15]:
clf = LogisticRegression(max_iter=1000)
clf.fit(x_train,y_train)
clf.score(x_train,y_train)

0.8676727909011374

In [16]:
print(precision_score(y_test,clf.predict(x_test)))
print(recall_score(y_test,clf.predict(x_test)))
print(f1_score(y_test,clf.predict(x_test)))

0.47297297297297297
0.2631578947368421
0.33816425120772947


In [17]:
clf = SVC()
clf.fit(x_train,y_train)
clf.score(x_train,y_train)

0.9431321084864392

In [18]:
print(precision_score(y_test,clf.predict(x_test)))
print(recall_score(y_test,clf.predict(x_test)))
print(f1_score(y_test,clf.predict(x_test)))

0.8611111111111112
0.23308270676691728
0.3668639053254438


In [19]:
df["v1"].value_counts()

0    4825
1     747
Name: v1, dtype: int64

In [22]:
ham = df[df["v1"]==0][:747]
spam = df[df["v1"]==1]

In [23]:
ham.shape,spam.shape

((747, 2), (747, 2))

In [29]:
df_under = pd.concat([spam,ham],axis=0)

In [34]:
a = np.random.permutation(df_under)

In [38]:
df_shuffle = pd.DataFrame(a,columns=["SpamHam","Message"])

In [42]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df_shuffle.Message)
X = tokenizer.texts_to_sequences(df_shuffle.Message)

In [44]:
X_padded = pad_sequences(X,maxlen=35)

In [45]:
y = df_shuffle.SpamHam.values

In [46]:
X_padded.shape,y.shape

((1494, 35), (1494,))

In [47]:
x_train,x_test,y_train,y_test = train_test_split(X_padded,y,test_size=.3)

In [55]:
y_train = np.array(y_train,dtype="int")
y_test = np.array(y_test,dtype="int")

In [79]:
import xgboost as xgb

In [80]:
# clf = LogisticRegression(max_iter=1000)
# clf.fit(x_train,y_train)

clf = xgb.XGBClassifier()
clf.fit(x_train,y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [81]:
clf.score(x_test,y_test)

0.8619153674832962

In [82]:
f1_score(y_test,clf.predict(x_test))

0.8577981651376146

In [83]:
precision_score(y_test,clf.predict(x_test))

0.8738317757009346

In [84]:
recall_score(y_test,clf.predict(x_test))

0.8423423423423423

In [85]:
from sklearn.metrics import precision_recall_curve